In [23]:
import pandas as pd

from bs4 import BeautifulSoup
import time
from selenium import webdriver

Use Selenium to open sourse.

In [14]:
driver = webdriver.Chrome()

In [15]:
url = 'https://www.8a.nu/crags/sportclimbing/turkey/geyik-bayiri/routes'

In [16]:
driver.get(url)

Create dataframe for saving parsing info.

In [102]:
data_container = pd.DataFrame(
    columns = ['grade',
               'name',
               'sector',
               'ascents',
               'fos_ratio',
               'recommendations',
               'stars']
)

data_container

,grade,name,sector,ascents,fos_ratio,recommendations,stars


Use BeautifulSoup for parsing.

In [24]:
# add our page into soup of html objects
html = driver.page_source
soup = BeautifulSoup(html)

In [103]:
# take whole table from page
elem_full_table = soup.find('table', class_='main-table zlags-table').find('tbody').find_all('tr')

In [104]:
# take one <tr> ond save routes info
for item in elem_full_table:
    grade = item.find('div', class_='grade').get_text()
    name = item.find('p', class_='name-link').get_text()
    sector = item.find('p', class_='sub-link').find_next('a').find_next('a').get_text()
    ascents = item.find(class_='col-ascents number').get_text()
    fos_ratio = item.find(class_='col-ratio').get_text()
    recommendations = item.find(class_='col-recommend').get_text()
    
    # count of stars of the route
    stars = item.find(class_='star-rating').find_all('label', class_='is-selected')

    new_row =\
        {'grade': grade.strip(),
         'name': name.strip(),
         'sector': sector.strip(),
         'ascents': ascents.strip(),
         'fos_ratio': fos_ratio.strip(),
         'recommendations': recommendations.strip(),
         'stars': len(stars)}
    
    # rewrite data in dataframe
    data_container = pd.concat(
        [data_container, pd.DataFrame([new_row])], ignore_index=True
    )

In [105]:
data_container

,grade,name,sector,ascents,fos_ratio,recommendations,stars
0,7a,Freedom is a Battle,Trebenna West,781,66 %,10 %,3
1,7a,Karınca,Magara,699,55 %,16 %,4
2,7b,Lycian Highway,Trebenna West,645,70 %,13 %,4
3,6b+,Saxafon,Sarkit,591,66 %,20 %,4
4,6b,Nirvana,Magara,589,78 %,10 %,3
5,7b+,Black Moon,Left Cave,568,36 %,20 %,4
6,6c+,Rocket Man,Left Cave,566,71 %,16 %,4
7,7a,Imagination,Sarkit,548,78 %,6 %,3
8,7b,Diplomarbeit,Trebenna West,516,30 %,17 %,4
9,7b,Rattle Snake Saloon,Trebenna West,515,42 %,10 %,3
